In [ ]:
import ee
import numpy as np
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=FfU-45tS47OX5YgLuMY7L-GVKRAJQFvV9ir1MD9alpM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6xX8ELQQLdFDxDRyftyC-0Xy_hrzDPDgDaGF3pfllyoBCb8oBFZgA

Successfully saved authorization token.


In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0MB 4.5MB/s 
     |████████████████████████████████| 6.5MB 11.7MB/s 
     |████████████████████████████████| 15.3MB 283kB/s 


In [ ]:
import geopandas as gpd
from datetime import date

dir_geojson = "drive/My Drive/GRASP_EARTH/world_bank/karachi.geojson"
dir_output = "GEE_output"
orbit = "ASCENDING"

start_date = date(2020, 7, 1).strftime('%Y-%m-%d')
end_date = date(2020, 10, 1).strftime('%Y-%m-%d')

In [ ]:
gdf = gpd.read_file(dir_geojson)
p = gdf["geometry"][0]
polygon = list(zip(*p.exterior.coords.xy))
aoi = ee.Geometry.Polygon(polygon)

In [ ]:
if orbit in ["ASCENDING", "DESCENDING"]:
    Sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
                .filterBounds(aoi)\
                .filterDate(start_date, end_date)\
                .filter(ee.Filter.eq('orbitProperties_pass', orbit))\
                .select(['VV'])
else:
    Sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
                .filterBounds(aoi)\
                .filterDate(start_date, end_date)\
                .select(['VV'])

img_list = Sentinel1.toList(100)

## 取得したデータ数の確認

In [ ]:
img_num = img_list.size().getInfo()

8

## Download image

In [ ]:
for i in range(img_list.size().getInfo()):
    image = ee.Image(img_list.get(i))
    task = ee.batch.Export.image.toDrive(
        image=image.reproject(crs='EPSG:4326',scale=10),
        description=image.get('system:index').getInfo(),
        folder=dir_output,
        region=aoi,
        scale=10
        )
    task.start()

processing:  ee.ComputedObject({
  "functionInvocationValue": {
    "functionName": "List.get",
    "arguments": {
      "index": {
        "constantValue": 0
      },
      "list": {
        "functionInvocationValue": {
          "functionName": "Collection.toList",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_0_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Image.select",
                          "arguments": {
                            "bandSelectors": {
                              "constantValue": [
                                "VV"
                              ]
           